The code imports various libraries and modules that will be used in the program. 

In [114]:
import pandas as pd
import requests 
import xlsxwriter 
import math
from bs4 import BeautifulSoup
import csv
import numpy as np
import time
from scipy import stats
from statistics import mean

The code is retrieving historical price data for a list of stock symbols from the Alpha Vantage API and performing calculations on the price returns for different time periods. Let's go through the code step by step:

First, an API key for Alpha Vantage is set. This key is used to authenticate and access the API.

The code reads a CSV file named 'S&P500.csv' using the Pandas library. The file contains stock ticker symbols under the column name 'Ticker'. These symbols are stored in the symbols list.

The interval variable is set to "DAILY", indicating that the historical price data will be retrieved on a daily basis.

The calculate_price_return(start_date, end_date) function is defined. It takes two parameters, start_date and end_date, representing the start and end dates for calculating the price return. Inside the function, it retrieves the closing prices for the specified dates from the prices dictionary, calculates the price return, and returns the result as a percentage.

An empty list df1 is initialized. This list will store dataframes for each stock symbol.

The code enters a loop that iterates over the symbols list in batches of four. This is done to avoid making too many API requests simultaneously.

Within the loop, the code makes an API request to retrieve historical price data for each stock symbol using the Alpha Vantage API. The URL for the request is constructed based on the symbol and the API key.

The response from the API request is obtained, and the JSON data is extracted.

The relevant price data is extracted from the JSON response and stored in the prices dictionary.

The code calculates the price returns for different time periods (3 years, 1 year, 6 months, 3 months, and 1 month) by calling the calculate_price_return function with the appropriate start and end dates. If an error occurs during the calculation, the corresponding price return is set to 0.

The current price is extracted from the prices dictionary using the fixed date "2023-05-12".

A new dataframe df is created to store the symbol, price, number of shares, price returns for different periods, and percentiles. The dataframe is appended to the df1 list.

After the loop finishes, the code concatenates all the dataframes in df1 into a single dataframe result_df using the pd.concat() function. The ignore_index=True argument ensures that the concatenated dataframe has a continuous index.

Finally, there is a time.sleep(60) statement, which pauses the execution for 60 seconds. This is likely added to avoid overwhelming the API with too many requests in a short period.

In [ ]:
# Set your Alpha Vantage API key
api_key = "API_KEY"

# Define the symbol and interval
stocks = pd.read_csv('Algorithm Trading/Equal weight S&P 500 index fund/S&P500.csv')
symbols = [stock for stock in stocks['Ticker']]
interval = "DAILY"

def calculate_price_return(start_date,end_date):
    price_start = float(prices[start_date]["4. close"])
    price_end = float(prices[end_date]["4. close"])
    price_return = ((price_end - price_start) / price_start) * 100
    return price_return

df1 = []

for i in range(0, len(symbols), 4):
    batch_symbols = symbols[i:i+4]
    try:
        for symbol in batch_symbols:
            # Make API request to retrieve historical price data
            url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&symbol={symbol}&apikey={api_key}"
            response = requests.get(url)
            data = response.json()

            # Extract the relevant price data for the desired time range (1 year)
            prices = data['Time Series (Daily)']

            try:
                # Calculate 3-year price return
                start_date_3yr = "2020-05-12"
                end_date_3yr = "2023-05-12"
                price_return_3yr = calculate_price_return(start_date_3yr, end_date_3yr)
            except:
                price_return_3yr = 0

            try:
                # Calculate 1-year price return
                start_date_1yr = "2022-05-12"
                end_date_1yr = "2023-05-12"
                price_return_1yr = calculate_price_return(start_date_1yr, end_date_1yr)
            except:
                price_return_1yr = 0

            try:
                # Calculate 6-month price return
                start_date_6mo = "2022-11-10"
                end_date_6mo = "2023-05-10"
                price_return_6mo = calculate_price_return(start_date_6mo, end_date_6mo)
            except:
                price_return_6mo = 0

            try:
                # Calculate 3-month price return
                start_date_3mo = "2023-02-10"
                end_date_3mo = "2023-05-10"
                price_return_3mo = calculate_price_return(start_date_3mo, end_date_3mo)
            except:
                price_return_3mo = 0

            try:
                # Calculate 1-month price return
                start_date_1mo = "2023-04-12"
                end_date_1mo = "2023-05-12"
                price_return_1mo = calculate_price_return(start_date_1mo, end_date_1mo)
            except:
                price_return_1mo = 0

            # Current Price
            price = float(prices["2023-05-12"]["4. close"])

            # Create dataframe for the symbol
            cols = ['Ticker', 'Price', 'Number of Shares to Buy', 'Three-Year Price Return', 'Three-Year Return Percentile',
                    'One-Year Price Return',
                    'One-Year Return Percentile',
                    'Six-Month Price Return',
                    'Six-Month Return Percentile',
                    'Three-Month Price Return',
                    'Three-Month Return Percentile',
                    'One-Month Price Return',
                    'One-Month Return Percentile',
                    'HQM Score']
            df = pd.DataFrame([[symbol, price, 0, price_return_3yr, 0, price_return_1yr, 0, price_return_6mo, 0, price_return_3mo, 0, price_return_1mo, 0, 0]], columns=cols)
            df1.append(df)
    except Exception as e:
        print(f"Error occurred for symbols: {batch_symbols}")
        print(f"Error message: {str(e)}")
    finally:
        result_df = pd.concat(df1,ignore_index = True)
    time.sleep(60)

The code is used to calculate the percentile ranks for different time periods (three-year, one-year, six-month, three-month, and one-month) based on the price returns of stocks.

The code iterates over each row in the result_df DataFrame, which is the DataFrame containing the stock data. It also iterates over the list lst which contains the names of the different time periods.

For each row and each time period, the code calculates the percentile rank of the corresponding price return value and assigns it to the respective column in the result_df DataFrame.

Here's a line-by-line explanation:

This line initializes a list called lst which contains the names of the different time periods.

Then it iterate over each row in the result_df DataFrame and each time period in the lst list. For each combination of row and time period, the code calculates the percentile rank of the corresponding price return value using the percentileofscore function from the stats module. The calculated percentile rank is then assigned to the respective column in the result_df DataFrame.

Overall, this code calculates the percentile ranks for the price returns of stocks for different time periods and adds them to the result_df DataFrame.

In [126]:
lst = ['Three-Year','One-Year','Six-Month','Three-Month','One-Month']
for row in range(len(result_df)):
    for year in lst:
        result_df.loc[row,f'{year} Return Percentile'] = stats.percentileofscore(result_df[f'{year} Price Return'], result_df.loc[row,f'{year} Price Return'], kind='mean')

Calculating HQM Score for Each Row:

The first part of the code calculates the "HQM Score" for each row in the result_df DataFrame. It iterates over the range of the length of the DataFrame, which means it goes through each row. For each row, it calculates the mean (average) of the "Return Percentile" values for each year in the lst list. It assigns this mean value to the "HQM Score" column of that row in the result_df DataFrame.

Sorting the DataFrame by HQM Score:

The second part of the code sorts the result_df DataFrame based on the values in the "HQM Score" column in descending order. This is done using the sort_values() function. The DataFrame is sorted in-place, which means it modifies the original DataFrame.

Selecting Top 50 Rows:

The third part of the code selects the top 50 rows from the sorted DataFrame and updates the result_df DataFrame with these selected rows. It assigns the updated DataFrame to the same variable result_df, overwriting the original DataFrame.

In [130]:
for row in range(len(result_df)):
     result_df.loc[row,'HQM Score'] = mean([result_df.loc[row,f'{year} Return Percentile'] for year in lst])
result_df.sort_values(by='HQM Score', ascending=False, inplace = True)
result_df = result_df[:50]

The code is responsible for calculating the number of shares to buy for each stock in a portfolio based on the portfolio size and the stock's current price.

Here's a breakdown of the code:

The code starts with a while loop that runs indefinitely until a valid number is entered for the portfolio size. It prompts the user to enter their portfolio size and attempts to convert the input to a float value using the float() function. If the conversion is successful (no ValueError is raised), the loop is exited using the break statement. If the conversion fails (a ValueError is raised), an error message is printed, indicating that a number should be entered. After exiting the loop, the variable val contains the portfolio size entered by the user. The code calculates the position size for each stock by dividing the portfolio size (val) by the number of stocks in the result_df DataFrame.

In [ ]:
while True:
    try:
        val = float(input("Enter your portfolio size: "))
        break  # Exit the loop if a valid number is entered
    except ValueError:
        print("Please enter a number.")
position_size = val/len(result_df)
for i in range(len(result_df)):
    result_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/float(result_df.loc[i, 'Price']))

The code is responsible for writing the data from the result_df DataFrame to a CSV file named "trades". It uses the csv module to accomplish this task.

Let's break down the code line by line:

with open('trades', 'w') as f: This line opens the file named "trades" in write mode ('w'). The file will be created if it doesn't exist, and if it exists, its contents will be overwritten. The file object is assigned to the variable f.

writer = csv.writer(f) This line creates a CSV writer object using the file object f as the output file. The csv.writer() function is used to create the writer.

In [ ]:
import csv

# Define the file path
file_path = "trades.csv"

# Write the header row
header_row = ['Ticker', 'Price', 'Number of Shares to Buy', 'Three-Year Price Return', 'Three-Year Return Percentile',
              'One-Year Price Return', 'One-Year Return Percentile', 'Six-Month Price Return',
              'Six-Month Return Percentile', 'Three-Month Price Return', 'Three-Month Return Percentile',
              'One-Month Price Return', 'One-Month Return Percentile', 'HQM Score']

with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header_row)

    for i in range(len(result_df)):
        ticker = result_df.loc[i, "Ticker"]
        price = result_df.loc[i, "Price"]
        num_shares_to_buy = result_df.loc[i, "Number of Shares to Buy"]
        three_year_return = result_df.loc[i, "Three-Year Price Return"]
        three_year_percentile = result_df.loc[i, "Three-Year Return Percentile"]
        one_year_return = result_df.loc[i, "One-Year Price Return"]
        one_year_percentile = result_df.loc[i, "One-Year Return Percentile"]
        six_month_return = result_df.loc[i, "Six-Month Price Return"]
        six_month_percentile = result_df.loc[i, "Six-Month Return Percentile"]
        three_month_return = result_df.loc[i, "Three-Month Price Return"]
        three_month_percentile = result_df.loc[i, "Three-Month Return Percentile"]
        one_month_return = result_df.loc[i, "One-Month Price Return"]
        one_month_percentile = result_df.loc[i, "One-Month Return Percentile"]
        hqm_score = result_df.loc[i, "HQM Score"]

        # Write data row
        writer.writerow([ticker, price, num_shares_to_buy, three_year_return, three_year_percentile,
                         one_year_return, one_year_percentile, six_month_return, six_month_percentile,
                         three_month_return, three_month_percentile, one_month_return, one_month_percentile,
                         hqm_score])
